In [ ]:
# Clone github repository
!git clone --branch federated https://github.com/AlessandroMaini/federated-learning-project.git

In [ ]:
%cd federated-learning-project 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import random
from data.cifar100_loader import get_cifar100_loaders
from models.prepare_model import get_frozen_dino_vits16_model
from tools.hyperparameter_tuning import run_grid_search
from eval import evaluate
from train import train
from sharding.dataset_division import dataset_division   
from FedAvg.averaging import averaging

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader, val_loader, test_loader, train_dataset = get_cifar100_loaders()
sharding_dataloaders, cardinality_datasets = dataset_division(train_dataset)
criterion = nn.CrossEntropyLoss()

In [ ]:
model = get_frozen_dino_vits16_model(device)

In [ ]:
start_round = 0
num_rounds = 2
best_test_acc = 0.0

hist_test_loss = []
hist_test_acc = []

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.95, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_rounds)

In [ ]:
for round in range(start_round, start_round + num_rounds):
    clientsIndex = list(range(100)) 
    random.shuffle(clientsIndex)
    chosenClients = [sharding_dataloaders[i] for i in clientsIndex[:10]] 
    chosenCardinality = [cardinality_datasets[i] for i in clientsIndex[:10]]

    epochs = 4 
    averaging(chosenClients, chosenCardinality, model, optimizer, device, epochs, criterion)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()

    hist_test_loss.append(test_loss)
    hist_test_acc.append(test_acc)

    print(f"Epoch {round+1}/{start_round + num_rounds}")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.4f}")

In [ ]:
# Plot the training and test accuracy
plt.plot(test_acc, label='Test Accuracy')
plt.xlabel('Rounds')
plt.ylabel('Accuracy')
plt.legend()
plt.show()